In [35]:
using JuMP
import LinearAlgebra    
import SCS
using Hypatia
using Ket

In [3]:
N, d = 2, 2

(2, 2)

In [4]:
function random_state(d)
    x = randn(ComplexF64, (d, d))
    y = x * x'
    return LinearAlgebra.Hermitian(y / LinearAlgebra.tr(y))
end

ρ = [random_state(d) for i in 1:N]
#ρ = 0.5 * [LinearAlgebra.Hermitian([[1,0] [0,1]])]
#ρ = 0.5 * [[[1,1] [1,1]],[[1,-1] [-1,1]]] # discriminate between plus and minus
#ρ = [Hermitian([1 0; 0 0]), 0.5 * Hermitian([1 1; 1 1])] # 0 and plus


ρ

2-element Vector{LinearAlgebra.Hermitian{ComplexF64, Matrix{ComplexF64}}}:
 [0.7425098865553261 + 0.0im -0.3760340210239671 + 0.2210600185552158im; -0.3760340210239671 - 0.2210600185552158im 0.2574901134446739 + 0.0im]
 [0.5647988421199441 + 0.0im 0.1314433967349732 + 0.02217642052904461im; 0.1314433967349732 - 0.02217642052904461im 0.435201157880056 + 0.0im]

In [5]:
model = Model(SCS.Optimizer)
set_silent(model)

In [6]:
E = [@variable(model, [1:d, 1:d] in HermitianPSDCone()) for i in 1:N]

2-element Vector{LinearAlgebra.Hermitian{GenericAffExpr{ComplexF64, VariableRef}, Matrix{GenericAffExpr{ComplexF64, VariableRef}}}}:
 [_[1] _[2] + _[4] im; _[2] - _[4] im _[3]]
 [_[5] _[6] + _[8] im; _[6] - _[8] im _[7]]

In [7]:
@constraint(model, sum(E) == LinearAlgebra.I)

[_[1] + _[5] - 1                  _[2] + _[6] + _[4] im + _[8] im
 _[2] + _[6] - _[4] im - _[8] im  _[3] + _[7] - 1] ∈ Zeros()

In [8]:
@objective(
    model,
    Max,
    sum(real(LinearAlgebra.tr(ρ[i] * E[i])) for i in 1:N) / N,
)

0.37125494327766306 _[1] - 0.3760340210239671 _[2] + 0.2210600185552158 _[4] + 0.12874505672233694 _[3] + 0.28239942105997207 _[5] + 0.1314433967349732 _[6] + 0.02217642052904461 _[8] + 0.217600578940028 _[7]

In [9]:
optimize!(model)
assert_is_solved_and_feasible(model)
solution_summary(model)

* Solver : SCS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 7.86646e-01
  Dual objective value : 7.86649e-01

* Work counters
  Solve time (sec)   : 8.31867e-04


In [10]:
objective_value(model)

0.7866461163004466

In [11]:
0.5 + 0.25 * sum(LinearAlgebra.svdvals(ρ[1] - ρ[2]))

0.7866484041528192

In [12]:
solution = [value.(e) for e in E]

2-element Vector{Matrix{ComplexF64}}:
 [0.6549891918985006 + 0.0im -0.4425921869500261 + 0.17345466733798648im; -0.4425921869500261 - 0.17345466733798648im 0.34501078844081623 + 0.0im]
 [0.3450107984063601 + 0.0im 0.44259218644371934 - 0.1734546668241809im; 0.44259218644371934 + 0.1734546668241809im 0.6549892009423599 + 0.0im]

## Solving the dual
$$\min\text{tr[K]}\\
\text{s.c. } K \geq q_i p_i,\quad i \in 1,\ldots,N

$$ 

In [13]:
model_dual = Model(() -> Hypatia.Optimizer(verbose = true))
#model_dual = Model(SCS.Optimizer)
set_silent(model_dual)

In [14]:
K =@variable(model_dual, [1:d, 1:d] in HermitianPSDCone())
for i in 1:N
    println(i)
    @constraint(model_dual, K >= (ρ[i]/N), HermitianPSDCone() )
    #@constraint(model_dual, K >= 0, HermitianPSDCone() )
    
end

1
2


In [15]:
model_dual

A JuMP Model
├ solver: Hypatia
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 4
├ num_constraints: 3
│ ├ Vector{VariableRef} in MOI.HermitianPositiveSemidefiniteConeTriangle: 1
│ └ Vector{AffExpr} in MOI.HermitianPositiveSemidefiniteConeTriangle: 2
└ Names registered in the model: none

In [16]:
@objective(
    model_dual,
    Min,
    real(LinearAlgebra.tr(K))
)

_[1] + _[3]

In [17]:
optimize!(model_dual)
assert_is_solved_and_feasible(model_dual)
solution_summary(model_dual)

* Solver : Hypatia

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 7.86648e-01
  Dual objective value : 7.86648e-01

* Work counters
  Solve time (sec)   : 5.68950e+00
  Barrier iterations : 9


In [18]:
println(objective_value(model))
objective_value(model_dual)

0.7866461163004466


0.7866484052877273

## Channel discrimination

In [ ]:

# Construct the Choi operator of an Amplitude Damping channel 
γ=67/100
# Write a Cell with the Kraus operators
# Since this map is Completely positive, left and right Choi operators are the same.
K0=[1 0; 0 sqrt(1-γ)]
K1 = [0 sqrt(γ); 0 0]
# Declare the Channels which will be used
C1= choi([K0,K1]);

# Construct the Choi operator of an Bit Flip channel 
η=87/100
K0=sqrt(η)* LinearAlgebra.I(2)
K1=sqrt(1-η)*[0 1; 1 0]

C2 = choi([K0,K1])

C = Array{ComplexF64}(undef, 4, 4, 2)  # Création d’un tableau 3D vide (2×2×2)

C[:, :, 1] = C1
C[:, :, 2] = C2

N=size(C,3); #Obtain the number of channels N
k=2; #Set the number of uses k equals 2



display(C)

4×4×2 Array{ComplexF64, 3}:
[:, :, 1] =
      1.0+0.0im  0.0+0.0im   0.0+0.0im  0.574456+0.0im
      0.0+0.0im  0.0+0.0im   0.0+0.0im       0.0+0.0im
      0.0+0.0im  0.0+0.0im  0.67+0.0im       0.0+0.0im
 0.574456+0.0im  0.0+0.0im   0.0+0.0im      0.33+0.0im

[:, :, 2] =
 0.87+0.0im   0.0+0.0im   0.0+0.0im  0.87+0.0im
  0.0+0.0im  0.13+0.0im  0.13+0.0im   0.0+0.0im
  0.0+0.0im  0.13+0.0im  0.13+0.0im   0.0+0.0im
 0.87+0.0im   0.0+0.0im   0.0+0.0im  0.87+0.0im

In [66]:
d=sqrt(size(C[:,:,1],1));
dIn=d;
dOut=d;
DIM=[d d d d];
p_i=ones(1,N)/N;
println(d ," ", p_i)

2.0 [0.5 0.5]


In [ ]:
model = Model(SCS.Optimizer)
T = [@variable(model, [1:Int(dIn^(2*k)), 1:Int(dOut^(2*k))] in HermitianPSDCone()) for i in 1:N]
for i=1:N
    pS = pS + trace(p_i(i)*T(:,:,i) * Tensor(C(:,:,i),k) );
    W = W + T(:,:,i);
end

2

2-element Vector{LinearAlgebra.Hermitian{GenericAffExpr{ComplexF64, VariableRef}, Matrix{GenericAffExpr{ComplexF64, VariableRef}}}}:
 [_[1] _[2] + _[137] im … _[106] + _[228] im _[121] + _[242] im; _[2] - _[137] im _[3] … _[107] + _[229] im _[122] + _[243] im; … ; _[106] - _[228] im _[107] - _[229] im … _[120] _[135] + _[256] im; _[121] - _[242] im _[122] - _[243] im … _[135] - _[256] im _[136]]
 [_[257] _[258] + _[393] im … _[362] + _[484] im _[377] + _[498] im; _[258] - _[393] im _[259] … _[363] + _[485] im _[378] + _[499] im; … ; _[362] - _[484] im _[363] - _[485] im … _[376] _[391] + _[512] im; _[377] - _[498] im _[378] - _[499] im … _[391] - _[512] im _[392]]